# Bengali Tokenizer using BPE(byte-pair-encoding) and WordPiece

In [3]:
import os
import json
import pandas as pd

## Loading dataset and taking only the bengali part from it

In [4]:
df1=pd.read_csv('/kaggle/input/bdnews24-corpus/bdnews24.csv')

In [5]:
df1.drop(labels=['date','url','en_category'],axis=1)

,title,bn_category,contents
0,কক্সবাজারে যুবকের গলাকাটা লাশ উদ্ধার,বাংলাদেশ,"কক্সবাজার, জুন ০১ (বিডিনিউজ টুয়েন্টিফোর ডটকম)-..."
1,"শ্রীলঙ্কা সফরে প্রধান কোচ শন উইলিয়ামস, সহকারী ...",ক্রিকেট,"ঢাকা, ২ জুন (বিডিনিউজ টুয়েন্টি ফোর ডট কম) --- ..."
2,রশিদুজ্জামান মিল্লাত ও তার স্ত্রী--পুত্রের বির...,বাংলাদেশ,"ঢাকা, জুন (বিডিনিউজ টুয়েন্টিফোর ডটকম)-- পাঁচ ক..."
3,কালো টাকা সাদা করতে দুমাস সময় ঘোষণা,বাংলাদেশ,"ঢাকা, জুন ০৪ (বিডিনিউজ টুয়েন্টিফোর ডটকম)-- মঙ্..."
4,আফ্রিকাকে ৬ উইকেটে হারিয়েছে এশিয়া,ক্রিকেট,"ব্যাঙ্গালোর, ৫ জুন (বিডিনিউজ টুয়েন্টি ফোর ডট ক..."
...,...,...,...
417803,কাশ্মীরে কারফিউ উঠে গেলে রক্ত ঝরবে: ইমরান,বিশ্ব,জাতিসংঘ সাধারণ পরিষদের অধিবেশনে শুক্রবারের ভাষ...
417804,রদ্রিগোকে এখনই ‘চাপ’ দিতে রাজি নন জিদান,খেলা,ওয়ান্দা মেত্রোপলিতানোয় আগামী শনিবার লা লিগার ম...
417805,অনির্দিষ্টকালের জন্য মাঠের বাইরে মেসি,খেলা,নতুন মৌসুমে দলে ফিরে গত ৫ অগাস্ট প্রথম অনুশীলন...
417806,"‘হাওয়া ভবনের’ কত টাকা সুইস ব্যাংকে, তাও দেখা হ...",রাজনীতি,শুক্রবার পূর্বাচলে ‘দুর্ঘটনা রোধে চালকদের উন্ন...


In [8]:
df=df1.copy()

## getting the data from the df to a text file

In [9]:
# Removing the na values, typecasting to string and converts it to a list(from a pandas series)

title_text = df['title'].dropna().astype(str).tolist()
contents_text = df['contents'].dropna().astype(str).tolist()

In [12]:
bengali_text = "\n".join(title_text + contents_text) # joining all the values and combine it to 

In [13]:
with open('bengali_text_for_bpe.txt', 'w', encoding='utf-8') as file:
    file.write(bengali_text)  # writing to file

In [18]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()

In [19]:
trainer = trainers.BpeTrainer(
    vocab_size=20000,   # vocab size             
    min_frequency=2,    # minimum frequency for the word to be included         
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

In [20]:
files = ["bengali_text_for_bpe.txt"]
tokenizer.train(files, trainer) 

In [21]:
tokenizer.save("bengali_bpe_tokenizer.json")

In [22]:
sample_text = "এটি একটি পরীক্ষামূলক বাক্য।"
encoded = tokenizer.encode(sample_text)
decoded = tokenizer.decode(encoded.ids)

In [26]:
print("Original:", sample_text)
print("Encoded IDs:", encoded.tokens)
print("Decoded:", decoded)

Original: এটি একটি পরীক্ষামূলক বাক্য।
Encoded IDs: ['Ġà¦ıà¦Ł', 'à¦¿', 'Ġà¦ıà¦ķà¦Ł', 'à¦¿', 'Ġà¦ªà¦°', 'à§Ģ', 'à¦ķ', 'à§į', 'à¦·', 'à¦¾', 'à¦®', 'à§Ĥ', 'à¦²à¦ķ', 'Ġà¦¬', 'à¦¾', 'à¦ķ', 'à§į', 'à¦¯', 'à¥¤']
Decoded:  এটি একটি পরীক্ষামূলক বাক্য।


In [33]:
for i, token_id in enumerate(encoded.ids):
    token = tokenizer.decode([token_id])
    print(f"Token {i+1}: {token}") #printing individual tokens

Token 1:  এট
Token 2: ি
Token 3:  একট
Token 4: ি
Token 5:  পর
Token 6: ী
Token 7: ক
Token 8: ্
Token 9: ষ
Token 10: া
Token 11: ম
Token 12: ূ
Token 13: লক
Token 14:  ব
Token 15: া
Token 16: ক
Token 17: ্
Token 18: য
Token 19: ।


## Word Piece instead of BPE

In [35]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
trainer = trainers.WordPieceTrainer(vocab_size=32000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.decoder = decoders.WordPiece()


In [36]:
files = ["/kaggle/working/bengali_text_for_bpe.txt"]
tokenizer.train(files, trainer)

In [37]:

tokenizer.save("bengali_wordpiece_tokenizer.json")


In [38]:
loaded_tokenizer = Tokenizer.from_file("bengali_wordpiece_tokenizer.json")
bengali_text = "এটি একটি পরীক্ষামূলক বাক্য।"

encoded = loaded_tokenizer.encode(bengali_text)

print("Tokens:")
for token in encoded.tokens:
    print(token)

print("\nToken IDs:", encoded.ids)

decoded = loaded_tokenizer.decode(encoded.ids)
print("\nDecoded text:", decoded)


Tokens:
Ġà¦ıà¦Ł
à¦¿
Ġà¦ıà¦ķà¦Ł
à¦¿
Ġà¦ªà¦°
à§Ģ
à¦ķ
à§į
à¦·
à¦¾
à¦®
à§Ĥ
à¦²à¦ķ
Ġà¦¬
à¦¾
à¦ķ
à§į
à¦¯
à¥¤

Token IDs: [703, 353, 498, 353, 429, 380, 362, 352, 393, 350, 374, 461, 576, 363, 350, 362, 352, 372, 397]

Decoded text: Ġà¦ıà¦Ł à¦¿ Ġà¦ıà¦ķà¦Ł à¦¿ Ġà¦ªà¦° à§Ģ à¦ķ à§į à¦· à¦¾ à¦® à§Ĥ à¦²à¦ķ Ġà¦¬ à¦¾ à¦ķ à§į à¦¯ à¥¤


In [39]:
bengali_words = bengali_text.split()
for word in bengali_words:
    encoded_word = loaded_tokenizer.encode(word)
    print(f"\nWord: {word}")
    print("Tokens:", encoded_word.tokens)


Word: এটি
Tokens: ['Ġà¦ıà¦Ł', 'à¦¿']

Word: একটি
Tokens: ['Ġà¦ıà¦ķà¦Ł', 'à¦¿']

Word: পরীক্ষামূলক
Tokens: ['Ġà¦ªà¦°', 'à§Ģ', 'à¦ķ', 'à§į', 'à¦·', 'à¦¾', 'à¦®', 'à§Ĥ', 'à¦²à¦ķ']

Word: বাক্য।
Tokens: ['Ġà¦¬', 'à¦¾', 'à¦ķ', 'à§į', 'à¦¯', 'à¥¤']
